# Unsupervised Learning: Clustering Lab





In [15]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cluster import AgglomerativeClustering, KMeans
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance

## 1. (50%) Implement the k-means clustering algorithm and the HAC (Hierarchical Agglomerative Clustering) algorithm.

### 1.1.1 HAC

### Code requirements 
- HAC should support both single link and complete link options.
- HAC automatically generates all clusterings from n to 1.  To simplify the amount of output you may want to implement a mechanism to specify for which k values actual output will be generated.


---
The output should include the following:
- The number of clusters (k).
- The total SSE of the full clustering. 


For each cluster report include:


- The centroid id.
- The number of instances tied to that centroid. 
* The SSE of that cluster. (The sum squared error (SSE) of a single cluster is the sum of the squared euclidean distance of each cluster member to the cluster centroid.)
---
You only need to handle continuous features

In [34]:
class HACClustering(BaseEstimator,ClassifierMixin):

    def __init__(self,k=3,link_type='single'): ## add parameters here
        """
        Args:
            k = how many final clusters to have
            link_type = single or complete. when combining two clusters use complete link or single link
        """
        self.link_type = link_type
        self.k = k

    def get_distances(self, clusters):
      distances = np.ones((len(clusters), len(clusters))) * np.inf
      for i in range(len(clusters)):
        for j in range(i+1, len(clusters)):
          # d = np.linalg.norm(clusters[i] - clusters[j], ord=2, axis=1)
          d = distance.cdist(clusters[i], clusters[j])

          if self.link_type == 'single':
            distances[i][j] = np.argmin(d)
          elif self.link_type == 'complete':
            d = np.where(d==np.inf, 0, d)
            distances[i][j] = np.argmax(d)

      return distances

    def fit(self,X,y=None):
        """ Fit the data; In this lab this will make the K clusters :D
        Args:
            X (array-like): A 2D numpy array with the training data
            y (array-like): An optional argument. Clustering is usually unsupervised so you don't need labels
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """

        num_clusters = X.shape[0]
        centroids = X[:self.k]
        clusters = []
        for x in X:
          x.shape = (1,X.shape[1])
          clusters.append(x)

        # print(len(clusters))
        while num_clusters > self.k:
          curr_distances = self.get_distances(clusters)

          ind = np.unravel_index(np.argmin(curr_distances, axis=None), curr_distances.shape)

          new_cluster = np.concatenate((clusters[ind[0]], clusters[ind[1]]), axis=0)
          del clusters[ind[1]]
          del clusters[ind[0]]
          clusters.append(new_cluster)
          
          num_clusters -= 1
        
        self.clusters = clusters

        return self

    def sse(self, centroid, cluster):
      distance = np.array([np.linalg.norm(cluster - centroid, ord=2, axis=1)])
      distance_squared = np.square(distance)
      return np.sum(distance_squared)

    def print_clusters(self):
        """
            Used for grading.
            print("{:d}\n".format(k))
            print("{:.4f}\n\n".format(total SSE))
            for each cluster and centroid:
                print(np.array2string(centroid,precision=4,separator=","))
                print("\n")
                print("{:d}\n".format(size of cluster))
                print("{:.4f}\n\n".format(SSE of cluster))
        """
        all_sse = []
        centroids = []

        for c in self.clusters:
          centroid = c.mean(0)
          centroids.append(centroid)

          all_sse.append(self.sse(centroid, c))

        all_sse = np.array(all_sse)
        print("{:d}\n".format(self.k))
        print("{:.4f}\n\n".format(np.sum(all_sse)))

        for i in range(len(self.clusters)):
          print(np.array2string(centroids[i],precision=4,separator=","))
          print("{:d}".format(self.clusters[i].shape[0]))
          print("{:.4f}\n\n".format(all_sse[i]))
      

In [4]:
def normalize(X):
  for i in range(X.shape[1]):
    X[:,i] = (X[:,i] - np.min(X[:,i])) / (np.max(X[:,i]) - np.min(X[:,i]))
  return X

### 1.1.2 Debug 

Debug your model by running it on the [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/abalone.arff)


---
The dataset was modified to be a lot smaller. The last datapoint should be on line 359 or the point 0.585,0.46,0.185,0.922,0.3635,0.213,0.285,10. The remaining points should be commented out.



- Normalize Data
- K = 5
- Use the first k instances as the initial centroids
- Use 4 decimal places and DO NOT ROUND when reporting SSE and centroid values.


---
Solutions in files:

[Debug HAC Single.txt](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/Debug%20HAC%20Single%20Link.txt)

[Debug HAC Complete.txt](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/Debug%20HAC%20Complete%20Link.txt)

In [35]:
## DEBUG SET ##

from scipy.io import arff
import pandas as pd

# Download file with curl
!curl https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/abalone.arff --output debug-training-dataset.arff

data = arff.loadarff('debug-training-dataset.arff')
df = pd.DataFrame(data[0])

data = np.array(df)
X = normalize(data)
# print(X.shape)

single_debug_HAC = HACClustering(5, 'single')
single_debug_HAC.fit(X)
print("SINGLE LINK*************************************")
single_debug_HAC.print_clusters()

complete_debug_HAC = HACClustering(5, 'complete')
complete_debug_HAC.fit(X)
print("\n\n\nCOMPLETE LINK********************************")
complete_debug_HAC.print_clusters()


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  187k  100  187k    0     0  2087k      0 --:--:-- --:--:-- --:--:-- 2087k
SINGLE LINK*************************************
5

58.9879


[0.6526,0.6438,0.5186,0.335 ,0.319 ,0.3506,0.2766,0.417 ]
34
9.9565


[0.7245,0.7224,0.614 ,0.388 ,0.3687,0.4355,0.301 ,0.4412]
24
2.8964


[0.5991,0.5961,0.5019,0.2841,0.2706,0.2939,0.2335,0.3824]
82
24.9125


[0.5489,0.538 ,0.4382,0.2533,0.2344,0.2403,0.2173,0.3853]
60
21.2225





COMPLETE LINK********************************
5

38.7344


[0.5093,0.4979,0.3961,0.1989,0.1973,0.2155,0.1589,0.2931]
60
12.6495


[0.5942,0.5883,0.4822,0.2586,0.2481,0.2681,0.2098,0.3843]
62
10.3419


[0.5238,0.5106,0.4241,0.2165,0.2159,0.2254,0.1689,0.3443]
34
8.1156


[0.828 ,0.8335,0.7213,0.5424,0.4861,0.5381,0.4667,0.5936]
44
7.6273




### 1.1.3 Evaluation

We will evaluate your model based on its print_clusters() output using [Evaluation Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/seismic-bumps_train.arff)

In [ ]:
# Load evaluation data

# Train on evaluation data

# Print clusters

### 1.2.1 K-Means

### Code requirements 
- Ability to choose k and specify k initial centroids
- Use Euclidean Distance as metric
- Ability to handle distance ties
- Include output label as a cluster feature


---
The output should include the following:
- The number of clusters (k).
- The total SSE of the full clustering. 


For each cluster report include:


- The centroid id.
- The number of instances tied to that centroid. 
- The SSE of that cluster. (The sum squared error (SSE) of a single cluster is the sum of the squared euclidean distance of each cluster member to the cluster centroid.)
---
You only need to handle continuous features

In [ ]:
class KMEANSClustering(BaseEstimator,ClusterMixin):

    def __init__(self,k=3,debug=False): ## add parameters here
        """
        Args:
            k = how many final clusters to have
            debug = if debug is true use the first k instances as the initial centroids otherwise choose random points as the initial centroids.
        """
        self.k = k
        self.debug = debug

    def fit(self,X,y=None):
        """ Fit the data; In this lab this will make the K clusters :D
        Args:
            X (array-like): A 2D numpy array with the training data
            y (array-like): An optional argument. Clustering is usually unsupervised so you don't need labels
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        return self
    
    def print_clusters(self):
        """
            Used for grading.
            print("{:d}\n".format(k))
            print("{:.4f}\n\n".format(total SSE))
            for each cluster and centroid:
                print(np.array2string(centroid,precision=4,separator=","))
                print("\n")
                print("{:d}\n".format(size of cluster))
                print("{:.4f}\n\n".format(SSE of cluster))
        """
        pass

### 1.2.2 Debug 

Debug your model by running it on the [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/abalone.arff)


- Train until convergence
- Normalize Data
- K = 5
- Use the first k instances as the initial centroids
- Use 4 decimal places and DO NOT ROUND when reporting SSE and centroid values


---
Solutions in files:

[Debug K Means.txt](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/Debug%20K%20Means.txt)

In [ ]:
# Load debug data

# Train on debug data

# Print clusters

### 1.2.3 Evaluation

We will evaluate your model based on its print_clusters() output using [Evaluation Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/seismic-bumps_train.arff)

In [ ]:
# Load evaluation data

# Train on evaluation data

# Print clusters

## 2.1.1 (7.5%) Clustering the Iris Classification problem - HAC

Load the Iris Dataset [Iris Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/iris.arff)

- Use single-link and complete link clustering algorithms
- State whether you normalize your data or not (your choice).  
- Show your results for clusterings using k = 2-7.  
- Graph the total SSE for each k and discuss your results (i.e. what kind of clusters are being made).
---

In [ ]:
# Iris Classification using single-link

In [ ]:
# Iris Classification using complete-link

Discuss differences between single-link and complete-link

## 2.1.2 (5%) Clustering the Iris Classification problem - HAC

Requirements:
- Repeat excercise 2.1.1 and include the output label as one of the input features.

In [ ]:
# Clustering Labels using single-link

In [ ]:
# Clustering Labels using complete-link

Discuss any differences between the results from 2.1.1 and 2.1.2.

## 2.2.1 (7.5%) Clustering the Iris Classification problem: K-Means

Load the Iris Dataset [Iris Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/iris.arff)

Run K-Means on the Iris dataset using the output label as a feature and without using the output label as a feature

Requirements:
- State whether you normalize your data or not (your choice).  
- Show your results for clusterings using k = 2-7.  
- Graph the total SSE for each k and discuss your results (i.e. what kind of clusters are being made).
---

In [ ]:
# Iris Classification without output label

In [ ]:
# Iris Classification with output label

Compare results and differences between using the output label and excluding the output label

## 2.2.2 (5%) Clustering the Iris Classification problem: K-Means

Requirements:
- Use the output label as an input feature
- Run K-Means 5 times with k=4, each time with different initial random centroids and discuss any variations in the results. 

In [ ]:
#K-Means 5 times

Discuss any variations in the results

## 3.1 (12.5%) Run the SK versions of HAC (both single and complete link) on iris including the output label and compare your results with those above.
Use the silhouette score for this iris problem(k = 2-7).  You may write your own code to do silhouette (optional extra credit) or you can use sklearn.metrics.silhouette_score. Please state if you coded your own silhouette score function to receive the extra credit points (described below). Discuss how helpful Silhouette appeared to be for selecting which clustering is best. You do not need to supply full Silhouette graphs, but you could if you wanted to.

Requirements
- Use the Sillhouette score for this iris problem (k= 2-7) 
- Use at least one other scoring function from [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) and compare the results. State which metric was used. 
- Possible sklean metrics include (* metrics require ground truth labels):
    - adjusted_mutual_info_score*
    - adjusted_rand_score*
    - homogeneity_score*
    - completeness_score*
    - fowlkes_mallows_score*
    - calinski_harabasz_score
    - davies_bouldin_score
- Experiment using different hyper-parameters. Discuss Results

In [ ]:
# Load sklearn



*Record impressions*

## 3.2 (12.5%) Run the SK version of k-means on iris including the output label and compare your results with those above. 

Use the silhouette score for this iris problem(k = 2-7). You may write your own code to do silhouette (optional extra credit) or you can use sklearn.metrics.silhouette_score. Please state if you coded your own silhouette score function to receive the extra credit points (described below). Discuss how helpful Silhouette appeared to be for selecting which clustering is best. You do not need to supply full Silhouette graphs, but you could if you wanted to.

Requirements
- Use the Sillhouette score for this iris problem (k= 2-7) 
- Use at least one other scoring function form sklearn.metrics and compare the results. State which metric was used
- Experiment different hyper-parameters. Discuss Results

In [ ]:
# Load sklearn 



*Record impressions*

## 4. (Optional 5% extra credit) For your silhouette experiment above, write and use your own code to calculate the silhouette scores, rather than the SK or other version. 


*Show findings here*

In [ ]:
# Copy function Below